# Performance of concat operator on cmip6 decadal

**Rooki** calls climate data operations on the **rook** processing service.

In [1]:
import os
import time
os.environ['ROOK_URL'] = 'http://rook.dkrz.de/wps'

from rooki import operators as ops

## Load intake catalog for cmip6 decadal

In [2]:
import intake

In [3]:
cat_url = "https://raw.githubusercontent.com/cp4cds/c3s_34g_manifests/master/intake/catalogs/c3s.yaml"

cat = intake.open_catalog(cat_url)
df_cmip6_decadal = cat['c3s-cmip6-decadal'].read()
df_cmip6_decadal.head()

/usr/local/Caskroom/mambaforge/base/envs/rooki/lib/python3.11/site-packages/dask/dataframe/io/csv.py:542: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


,ds_id,path,size,mip_era,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,start_time,end_time,bbox,level,member_year,member_ensemble,prefix
0,c3s-cmip6-decadal.DCPP.MPI-M.MPI-ESM1-2-HR.dcp...,DCPP/MPI-M/MPI-ESM1-2-HR/dcppA-hindcast/s2016-...,492479692,c3s-cmip6-decadal,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,s2016-r8i1p1f1,day,tasmax,gn,v20210111,2016-11-01T12:00:00,2026-12-31T12:00:00,"0.00, -89.28, 359.06, 89.28",2.00,s2016,r8i1p1f1,c3s-cmip6-decadal
1,c3s-cmip6-decadal.DCPP.MPI-M.MPI-ESM1-2-HR.dcp...,DCPP/MPI-M/MPI-ESM1-2-HR/dcppA-hindcast/s2009-...,737204109,c3s-cmip6-decadal,DCPP,MPI-M,MPI-ESM1-2-HR,dcppA-hindcast,s2009-r8i1p1f1,day,pr,gn,v20210107,2009-11-01T12:00:00,2019-12-31T12:00:00,"0.00, -89.28, 359.06, 89.28",NaN,s2009,r8i1p1f1,c3s-cmip6-decadal
2,c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dc...,DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1987...,953384,c3s-cmip6-decadal,DCPP,MOHC,HadGEM3-GC31-MM,dcppA-hindcast,s1987-r3i1p1f2,Amon,pr,gn,v20200417,1987-11-16T00:00:00,1987-12-16T00:00:00,"0.42, -89.72, 359.58, 89.72",NaN,s1987,r3i1p1f2,c3s-cmip6-decadal
3,c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dc...,DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s1995...,603757,c3s-cmip6-decadal,DCPP,MOHC,HadGEM3-GC31-MM,dcppA-hindcast,s1995-r6i1p1f2,Amon,tas,gn,v20200417,1995-11-16T00:00:00,1995-12-16T00:00:00,"0.42, -89.72, 359.58, 89.72",1.50,s1995,r6i1p1f2,c3s-cmip6-decadal
4,c3s-cmip6-decadal.DCPP.MOHC.HadGEM3-GC31-MM.dc...,DCPP/MOHC/HadGEM3-GC31-MM/dcppA-hindcast/s2004...,14824913,c3s-cmip6-decadal,DCPP,MOHC,HadGEM3-GC31-MM,dcppA-hindcast,s2004-r10i1p1f2,AERday,zg500,gn,v20200417,2004-11-01T12:00:00,2004-12-30T12:00:00,"0.42, -89.72, 359.58, 89.72",50000.00,s2004,r10i1p1f2,c3s-cmip6-decadal


In [4]:
df = df_cmip6_decadal.loc[
    (df_cmip6_decadal.variable_id=="pr") 
    & (df_cmip6_decadal.experiment_id=="dcppB-forecast")
    & (df_cmip6_decadal.table_id=="Amon")
    & (df_cmip6_decadal.institution_id=="CMCC")
    & (df_cmip6_decadal.member_year=="s2020")
]
dsets = list(df.ds_id.unique())
dsets

['c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r10i1p1f1.Amon.pr.gn.v20210324',
 'c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r1i1p1f1.Amon.pr.gn.v20210324',
 'c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r2i1p1f1.Amon.pr.gn.v20210324',
 'c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r3i1p1f1.Amon.pr.gn.v20210324',
 'c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r4i1p1f1.Amon.pr.gn.v20210324',
 'c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r5i1p1f1.Amon.pr.gn.v20210324',
 'c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r6i1p1f1.Amon.pr.gn.v20210324',
 'c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r7i1p1f1.Amon.pr.gn.v20210324',
 'c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r8i1p1f1.Amon.pr.gn.v20210324',
 'c3s-cmip6-decadal.DCPP.CMCC.CMCC-CM2-SR5.dcppB-forecast.s2020-r9i1p1f1.Amon.pr.gn.v20210324']

## Run concat

https://docs.xarray.dev/en/stable/generated/xarray.concat.html

In [5]:
start = time.time()

t = "2020/2021" 

wf = ops.Concat(
        ops.Input('ds', dsets),
        time=t,
        dims="realization"
)

# optional average
wf = ops.Average(wf, dims="realization")

# optional subset
wf = ops.Subset(wf, time=t, time_components="month:nov,dec")

resp = wf.orchestrate()

end = time.time()
print(f"duration: {int(end - start)} secs")
print(resp)

resp.ok

duration: 9 secs
Metalink URL: http://rook4.cloud.dkrz.de:80/outputs/rook/ba52bb8c-104b-11ee-ab57-fa163ed6c06f/input.meta4, num files: 1


True

**open with xarray**

In [6]:
resp.size_in_mb

0.8846540451049805

In [7]:
resp.download_urls()

['http://rook4.cloud.dkrz.de:80/outputs/rook/beb1bb74-104b-11ee-b7bf-fa163ed6c06f/pr_Amon_CMCC-CM2-SR5_dcppB-forecast_r10i1p1f1_gn_20201116-20211216.nc']

In [8]:
dsets = resp.datasets()

In [9]:
ds = dsets[0]
ds

<xarray.Dataset>
Dimensions:      (time: 4, lat: 192, bnds: 2, lon: 288)
Coordinates:
  * time         (time) object 2020-11-16 12:00:00 ... 2021-12-16 00:00:00
  * lat          (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon          (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    reftime      object ...
    leadtime     (time) timedelta64[ns] ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds     (lat, bnds) float64 ...
    lon_bnds     (lon, bnds) float64 ...
    pr           (time, lat, lon) float32 ...
    realization  float64 ...
Attributes: (12/52)
    Conventions:                 CF-1.7 CMIP-6.2
    activity_id:                 DCPP
    branch_method:               no parent
    branch_time_in_child:        0.0
    branch_time_in_parent:       0.0
    comment:                     none
    ...                          ...
    cmor_version:                3.6.0
    tracking_id:                 hdl:21.14100/a12ca499-1524-4b77-8d82-99d71f6...
    forcing_description:         f1, CMIP6 historical forcings
    physics_description:         physics from the standard model configuratio...
    initialization_description:  hindcast initialized based on observations a...
    startdate:                   s202011

**provenance**

In [10]:
prov_plot_url = resp.provenance_image()
prov_plot_url

'http://rook4.cloud.dkrz.de:80/outputs/rook/ba52bb8c-104b-11ee-ab57-fa163ed6c06f/provenance.png'

In [11]:
# from IPython.display import Image
# Image(prov_plot_url)

In [12]:
resp.provenance()

'http://rook4.cloud.dkrz.de:80/outputs/rook/ba52bb8c-104b-11ee-ab57-fa163ed6c06f/provenance.json'